In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('/kaggle/input/titanic/train.csv')

In [3]:
df1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df1.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'], axis=1, inplace=True )

In [5]:
df1.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [6]:
df1.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
Fare          0
dtype: int64

In [7]:
df1['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [8]:
df1.fillna({'Age':df1['Age'].mean()},inplace=True)

In [9]:
df1.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
Fare        0
dtype: int64

In [10]:
df1.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [11]:
pd.set_option('future.no_silent_downcasting', True)

In [12]:
df1.replace({'Sex':{'male':0,'female':1}}, inplace=True)

In [13]:
df1.rename(columns={'Sex': 'male'}, inplace=True)

In [14]:
df1.head()

,Survived,Pclass,male,Age,Fare
0,0,3,0,22.0,7.2500
1,1,1,1,38.0,71.2833
2,1,3,1,26.0,7.9250
3,1,1,1,35.0,53.1000
4,0,3,0,35.0,8.0500


In [15]:
from sklearn.preprocessing import StandardScaler
sts =StandardScaler()

In [16]:
feature_scale = ['Age','Fare']
df1[feature_scale] = sts.fit_transform(df1[feature_scale])

In [17]:
df1.head()

,Survived,Pclass,male,Age,Fare
0,0,3,0,-0.592481,-0.502445
1,1,1,1,0.638789,0.786845
2,1,3,1,-0.284663,-0.488854
3,1,1,1,0.407926,0.420730
4,0,3,0,0.407926,-0.486337


In [18]:
df1.rename(columns={'Sex': 'male'}, inplace=True)

In [19]:
df1.head()

,Survived,Pclass,male,Age,Fare
0,0,3,0,-0.592481,-0.502445
1,1,1,1,0.638789,0.786845
2,1,3,1,-0.284663,-0.488854
3,1,1,1,0.407926,0.420730
4,0,3,0,0.407926,-0.486337


In [20]:
X=df1.drop(['Survived'],axis=1)
y=df1['Survived']

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [22]:
#create param
model_param = {
    'DecisionTreeClassifier':{
        'model':DecisionTreeClassifier(),
        'param':{
            'criterion': ['gini','entropy']
        }
    },
        'KNeighborsClassifier':{
        'model':KNeighborsClassifier(),
        'param':{
            'n_neighbors': [5,10,15,20,25]
        }
    },
        'SVC':{
        'model':SVC(),
        'param':{
            'kernel':['rbf','linear','sigmoid'],
            'C': [0.1, 1, 10, 100]
         
        }
    }
}

In [23]:
scores =[]
for model_name, mp in model_param.items():
    model_selection = GridSearchCV(estimator=mp['model'],param_grid=mp['param'],cv=5,return_train_score=False)
    model_selection.fit(X,y)
    scores.append({
        'model': model_name,
        'best_score': model_selection.best_score_,
        'best_params': model_selection.best_params_
    })

In [24]:
df_model_score = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df_model_score

,model,best_score,best_params
0,DecisionTreeClassifier,0.776718,{'criterion': 'gini'}
1,KNeighborsClassifier,0.803616,{'n_neighbors': 5}
2,SVC,0.811481,"{'C': 100, 'kernel': 'rbf'}"


In [25]:
model_svc = SVC( C= 100,kernel='rbf')

In [26]:
model_svc.fit(X, y)

SVC(C=100)

In [27]:
df2 = pd.read_csv('/kaggle/input/titanic/test.csv')

In [28]:
df2.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [29]:
df3=df2.drop(['PassengerId','Name','Ticket','Cabin','Embarked','SibSp','Parch'], axis=1 )

In [30]:
df3.isnull().sum()

Pclass     0
Sex        0
Age       86
Fare       1
dtype: int64

In [31]:
df3.fillna({'Age':df1['Age'].mean()},inplace=True)
df3.fillna({'Fare':df1['Fare'].mean()},inplace=True)

In [32]:
df3.replace({'Sex':{'male':0,'female':1}}, inplace=True)

In [33]:
df3.rename(columns={'Sex': 'male'}, inplace=True)

In [34]:
df3

,Pclass,male,Age,Fare
0,3,0,3.450000e+01,7.8292
1,3,1,4.700000e+01,7.0000
2,2,0,6.200000e+01,9.6875
3,3,0,2.700000e+01,8.6625
4,3,1,2.200000e+01,12.2875
...,...,...,...,...
413,3,0,2.232906e-16,8.0500
414,1,1,3.900000e+01,108.9000
415,3,0,3.850000e+01,7.2500
416,3,0,2.232906e-16,8.0500


In [35]:
df3[feature_scale] = sts.fit_transform(df3[feature_scale])

In [36]:
df3.head()

,Pclass,male,Age,Fare
0,3,0,0.594801,-0.496637
1,3,1,1.305896,-0.511497
2,2,0,2.159209,-0.463335
3,3,0,0.168145,-0.481704
4,3,1,-0.116293,-0.416740


In [38]:
y_predicted = model_svc.predict(df3)

In [39]:
submission = pd.DataFrame({
        "PassengerId": df2['PassengerId'],
        "Survived": y_predicted
    })

In [41]:
submission.to_csv('titanic_submission.csv', index=False)

In [45]:
df4=pd.read_csv('titanic_submission.csv')

In [46]:
df4

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
